# Notes

February 29th in 2020. I am taking that day out. because it's not in the NOAA data and 2020 had a leap year. 

# Goal

I want to automate the CAISO data. Each csv's column name should be its year and date + megawatt.
Ex: 2020-01-01Megawatts 
or something like that 

# Result

This script takes out hour-ahead forecast, solar, and wind, leaving ONLY demand. Then the demand energy is summed up by hour and put into a file. Thus this code produces hourly energy usage per file. 

In [120]:
import pandas as pd
import numpy as np
import datetime
import os
from os import listdir
from os.path import isfile, join

In [121]:
# df = pd.read_csv("/Users/yukahatori/A_Fairness/FairnessML_git/CAISO-netdemand-20200101.csv", on_bad_lines = 'skip')
# df
weirdOne = pd.read_csv("/Users/yukahatori/A_Fairness/FairnessML_git/CAISO-netdemand-20200311.csv", on_bad_lines = 'skip')
print("col elements when there is the weird one: ",len(weirdOne.columns))
weirdOne.drop('Unnamed: 290', axis=1, inplace= True)
weirdOne
normalOne = pd.read_csv("/Users/yukahatori/A_Fairness/FairnessML_git/CAISO-netdemand-20200101.csv", on_bad_lines = 'skip')
print("col elements when there is no weird one: ",len(normalOne.columns))

print("weird one has 5 rows, non-weird one only has 4")

print("Non-weird one")
normalOne

col elements when there is the weird one:  291
col elements when there is no weird one:  290
weird one has 5 rows, non-weird one only has 4
Non-weird one


,Net Demand 01/01/2020,00:00,00:05,00:10,00:15,00:20,00:25,00:30,00:35,00:40,...,23:15,23:20,23:25,23:30,23:35,23:40,23:45,23:50,23:55,00:00.1
0,Hour-ahead forecast,21474,21168,21168,21168,21015,21015,21015,20755,20755,...,20967,20655,20655,20655,20321,20321,20321,20078,20078,20078.0
1,Demand,21533,21429,21320,21272,21193,21109,21006,20961,20916,...,20920,20809,20681,20574,20494,20383,20297,20242,20128,20025.0
2,Solar,-33,-33,-33,-33,-33,-33,-33,-33,-33,...,-33,-33,-33,-33,-33,-33,-33,-33,-33,NaN
3,Wind,808,804,800,790,790,792,785,787,793,...,3248,3267,3307,3339,3404,3451,3482,3472,3455,NaN


In [122]:
print("Weird one")
weirdOne

Weird one


,Net Demand 03/11/2020,00:00,00:05,00:10,00:15,00:20,00:25,00:30,00:35,00:40,...,23:15,23:20,23:25,23:30,23:35,23:40,23:45,23:50,23:55,00:00.1
0,Hour-ahead forecast,21924,21656,21656,21656,21432,21432,21432,21200,21200,...,22944,22563,22563,22563,22227,22227,22227,21852,21852,21852.0
1,Demand,21809,21759,21723,21659,21501,21352,21333,21311,21219,...,22721,22600,22458,22337,22216,21916,21820,21791,21698,21578.0
2,Day-ahead net forecast,Net demand,21387,21292,21320,21248,21185,20917,20858,20885,...,21821,21680,21549,21409,21274,21129,20829,20707,20658,20563.0
3,Solar,-29,-29,-29,-29,-29,-29,-29,-29,-28,...,-27,-27,-27,-27,-27,-27,-27,-27,-27,NaN
4,Wind,506,493,470,464,468,486,495,488,469,...,1069,1083,1082,1088,1116,1126,1138,1158,1166,NaN


In [123]:
normalOne.drop(labels = [0,2, 3], axis = 0)
weirdOne.drop(labels = [0,2, 3,4], axis = 0)

,Net Demand 03/11/2020,00:00,00:05,00:10,00:15,00:20,00:25,00:30,00:35,00:40,...,23:15,23:20,23:25,23:30,23:35,23:40,23:45,23:50,23:55,00:00.1
1,Demand,21809,21759,21723,21659,21501,21352,21333,21311,21219,...,22721,22600,22458,22337,22216,21916,21820,21791,21698,21578.0


In [124]:
normalOne = normalOne.drop(labels = [0,2,3], axis = 0)
weirdOne = weirdOne.drop(labels = [0,2,3,4], axis = 0)

In [125]:
print(normalOne.columns[278:290])

def addHours(df,year, month, day):
    start = 1
    end = 14
    for hour in range(24):
        x = datetime.datetime(year, month, day, hour)
        df[x] = df.iloc[:,start:end].sum(axis = 1)
        start = end
        end +=12
    return df

Index(['23:05', '23:10', '23:15', '23:20', '23:25', '23:30', '23:35', '23:40',
       '23:45', '23:50', '23:55', '00:00.1'],
      dtype='object')


In [126]:
normalOne = addHours(normalOne, 2020, 1, 1)
weirdOne = addHours(weirdOne, 2020, 3, 11)

/var/folders/yv/7r0htvks26q7nhdm7l86zm7m0000gn/T/ipykernel_12185/4252209769.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df[x] = df.iloc[:,start:end].sum(axis = 1)


In [127]:
def removeOldDates(df, firstVal):
    df = df.drop(columns = [firstVal,'00:00.1','00:00','00:05','00:10','00:15','00:20','00:25','00:30','00:35','00:40','00:45','00:50','00:55','01:00','01:05','01:10','01:15','01:20','01:25','01:30','01:35','01:40','01:45','01:50','01:55','02:00','02:05','02:10','02:15','02:20','02:25','02:30','02:35','02:40','02:45','02:50','02:55','03:00','03:05','03:10','03:15','03:20','03:25','03:30','03:35','03:40','03:45','03:50','03:55','04:00','04:05','04:10','04:15','04:20','04:25','04:30','04:35','04:40','04:45','04:50','04:55','05:00','05:05','05:10','05:15','05:20','05:25','05:30','05:35','05:40','05:45','05:50','05:55','06:00','06:05','06:10','06:15','06:20','06:25','06:30','06:35','06:40','06:45','06:50','06:55','07:00','07:05','07:10','07:15','07:20','07:25','07:30','07:35','07:40','07:45','07:50','07:55','08:00','08:05','08:10','08:15','08:20','08:25','08:30','08:35','08:40','08:45','08:50','08:55','09:00','09:05','09:10','09:15','09:20','09:25','09:30','09:35','09:40','09:45','09:50','09:55','10:00','10:05','10:10','10:15','10:20','10:25','10:30','10:35','10:40','10:45','10:50','10:55','11:00','11:05','11:10','11:15','11:20','11:25','11:30','11:35','11:40','11:45','11:50','11:55','12:00','12:05','12:10','12:15','12:20','12:25','12:30','12:35','12:40','12:45','12:50','12:55','13:00','13:05','13:10','13:15','13:20','13:25','13:30','13:35','13:40','13:45','13:50','13:55','14:00','14:05','14:10','14:15','14:20','14:25','14:30','14:35','14:40','14:45','14:50','14:55','15:00','15:05','15:10','15:15','15:20','15:25','15:30','15:35','15:40','15:45','15:50','15:55','16:00','16:05','16:10','16:15','16:20','16:25','16:30','16:35','16:40','16:45','16:50','16:55','17:00','17:05','17:10','17:15','17:20','17:25','17:30','17:35','17:40','17:45','17:50','17:55','18:00','18:05','18:10','18:15','18:20','18:25','18:30','18:35','18:40','18:45','18:50','18:55','19:00','19:05','19:10','19:15','19:20','19:25','19:30','19:35','19:40','19:45','19:50','19:55','20:00','20:05','20:10','20:15','20:20','20:25','20:30','20:35','20:40','20:45','20:50','20:55','21:00','21:05','21:10','21:15','21:20','21:25','21:30','21:35','21:40','21:45','21:50','21:55','22:00','22:05','22:10','22:15','22:20','22:25','22:30','22:35','22:40','22:45','22:50','22:55','23:00','23:05','23:10','23:15','23:20','23:25','23:30','23:35','23:40','23:45','23:50','23:55'])
    return df


In [129]:
normalOne = removeOldDates(normalOne, 'Net Demand 01/01/2020')
normalOne

,2020-01-01 00:00:00,2020-01-01 01:00:00,2020-01-01 02:00:00,2020-01-01 03:00:00,2020-01-01 04:00:00,2020-01-01 05:00:00,2020-01-01 06:00:00,2020-01-01 07:00:00,2020-01-01 08:00:00,2020-01-01 09:00:00,...,2020-01-01 14:00:00,2020-01-01 15:00:00,2020-01-01 16:00:00,2020-01-01 17:00:00,2020-01-01 18:00:00,2020-01-01 19:00:00,2020-01-01 20:00:00,2020-01-01 21:00:00,2020-01-01 22:00:00,2020-01-01 23:00:00
1,273786,243000,235962,231944,230219,235383,243281,243793,237974,228866,...,227833,242087,263675,294908,299952,296337,289061,278538,262639,246686.0


In [128]:

weirdOne = removeOldDates(weirdOne, 'Net Demand 03/11/2020')
weirdOne

,2020-03-11 00:00:00,2020-03-11 01:00:00,2020-03-11 02:00:00,2020-03-11 03:00:00,2020-03-11 04:00:00,2020-03-11 05:00:00,2020-03-11 06:00:00,2020-03-11 07:00:00,2020-03-11 08:00:00,2020-03-11 09:00:00,...,2020-03-11 14:00:00,2020-03-11 15:00:00,2020-03-11 16:00:00,2020-03-11 17:00:00,2020-03-11 18:00:00,2020-03-11 19:00:00,2020-03-11 20:00:00,2020-03-11 21:00:00,2020-03-11 22:00:00,2020-03-11 23:00:00
1,255868,246337,240353,237398,242124,259185,288761,308108,304023,294490,...,262293,267238,276641,292529,309148,327959,324322,309974,288160,267015.0


In [130]:
normalOne = normalOne.T
normalOne

,1
2020-01-01 00:00:00,273786.0
2020-01-01 01:00:00,243000.0
2020-01-01 02:00:00,235962.0
2020-01-01 03:00:00,231944.0
2020-01-01 04:00:00,230219.0
2020-01-01 05:00:00,235383.0
2020-01-01 06:00:00,243281.0
2020-01-01 07:00:00,243793.0
2020-01-01 08:00:00,237974.0
2020-01-01 09:00:00,228866.0


In [133]:
weirdOne = weirdOne.T
weirdOne

,1
2020-03-11 00:00:00,255868.0
2020-03-11 01:00:00,246337.0
2020-03-11 02:00:00,240353.0
2020-03-11 03:00:00,237398.0
2020-03-11 04:00:00,242124.0
2020-03-11 05:00:00,259185.0
2020-03-11 06:00:00,288761.0
2020-03-11 07:00:00,308108.0
2020-03-11 08:00:00,304023.0
2020-03-11 09:00:00,294490.0


In [134]:
normalOne.rename(columns = {1: 'Megawatts'}, inplace= True)
normalOne['date'] = normalOne.index
normalOne = normalOne[['date', 'Megawatts']]

In [135]:
normalOne

,date,Megawatts
2020-01-01 00:00:00,2020-01-01 00:00:00,273786.0
2020-01-01 01:00:00,2020-01-01 01:00:00,243000.0
2020-01-01 02:00:00,2020-01-01 02:00:00,235962.0
2020-01-01 03:00:00,2020-01-01 03:00:00,231944.0
2020-01-01 04:00:00,2020-01-01 04:00:00,230219.0
2020-01-01 05:00:00,2020-01-01 05:00:00,235383.0
2020-01-01 06:00:00,2020-01-01 06:00:00,243281.0
2020-01-01 07:00:00,2020-01-01 07:00:00,243793.0
2020-01-01 08:00:00,2020-01-01 08:00:00,237974.0
2020-01-01 09:00:00,2020-01-01 09:00:00,228866.0


In [136]:
weirdOne.rename(columns = {1: 'Megawatts'}, inplace= True)
weirdOne['date'] = weirdOne.index
weirdOne = weirdOne[['date', 'Megawatts']]

In [137]:
normalOne.to_csv('/Users/yukahatori/A_Fairness/FairnessML_git/CAISO_test_normal.csv', encoding='utf-8', index=False)

In [138]:
weirdOne.to_csv('/Users/yukahatori/A_Fairness/FairnessML_git/CAISO_test_weird.csv', encoding='utf-8', index=False)

# Above, was an example of processing one CAISO file. Below, the process is generalized to all files

In [141]:
def processCaisoFiles(working_directory, destination_directory):
    os.chdir(working_directory)
    allDataFiles = [f for f in listdir(working_directory) if isfile(join(working_directory, f))] #returns the file names in that directory
    for fileName in allDataFiles:
        print(fileName)
        date = fileName[16:24]
        df = pd.read_csv(fileName, on_bad_lines = 'skip')
        if len(df.columns) == 290: #then it's "normal"
            df = df.drop(labels = [0,2,3], axis = 0)
        elif len(df.columns) == 291: #then it's "weird"
            df.drop('Unnamed: 290', axis=1, inplace=True)
            df = df.drop(labels = [0,2,3,4], axis = 0)
        else:
            print("something is weird.")
            return 
        # year, month, day
        #0:4 = year; 4:6 = month; 6:8 = day
        df = addHours(df, int(date[0:4]), int(date[4:6]),int(date[6:8]))
        # month/day/year
        colDate = date[4:6] +'/'+ date[6:8] +'/'+ date[0:4]
        extraCol = 'Net Demand ' + colDate
        df = removeOldDates(df, extraCol) #there's one weird column called net demand <date> that I want to remove
        df = df.T
        renameCol = 'Megawatts'
        df.rename(columns = {1: renameCol}, inplace= True)
        df['date'] = df.index
        df = df[['date', 'Megawatts']]
        newCsv = destination_directory + 'CAISO-netdemand-' + date +'-Megawatts.csv'
        df.to_csv(newCsv, encoding='utf-8', index = False)
        

In [142]:
wd = '/Users/yukahatori/A_Fairness/FairnessML_git/CAISO_NetDemand_Clean'
destination = '/Users/yukahatori/A_Fairness/FairnessML_git/CAISO_NetDemand_Megawatts/'
processCaisoFiles(wd, destination)

CAISO-netdemand-20190609.csv
CAISO-netdemand-20190621.csv
CAISO-netdemand-20210927.csv
CAISO-netdemand-20180426.csv
CAISO-netdemand-20190806.csv
CAISO-netdemand-20200513.csv
CAISO-netdemand-20200507.csv
CAISO-netdemand-20210714.csv
CAISO-netdemand-20190812.csv
CAISO-netdemand-20210728.csv
CAISO-netdemand-20180618.csv
CAISO-netdemand-20180630.csv
CAISO-netdemand-20190423.csv
CAISO-netdemand-20180624.csv
CAISO-netdemand-20200922.csv
CAISO-netdemand-20210502.csv
CAISO-netdemand-20200711.csv
CAISO-netdemand-20180817.csv
CAISO-netdemand-20180803.csv
CAISO-netdemand-20210516.csv
CAISO-netdemand-20200705.csv
CAISO-netdemand-20191113.csv
CAISO-netdemand-20220210.csv
CAISO-netdemand-20220204.csv


/var/folders/yv/7r0htvks26q7nhdm7l86zm7m0000gn/T/ipykernel_12185/4252209769.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df[x] = df.iloc[:,start:end].sum(axis = 1)


CAISO-netdemand-20191107.csv
CAISO-netdemand-20190219.csv
CAISO-netdemand-20190225.csv
CAISO-netdemand-20210310.csv
CAISO-netdemand-20200103.csv
CAISO-netdemand-20201209.csv
CAISO-netdemand-20210304.csv
CAISO-netdemand-20200117.csv
CAISO-netdemand-20201221.csv
CAISO-netdemand-20211026.csv
CAISO-netdemand-20181102.csv
CAISO-netdemand-20181116.csv
CAISO-netdemand-20200301.csv
CAISO-netdemand-20210112.csv
CAISO-netdemand-20200315.csv
CAISO-netdemand-20210106.csv
CAISO-netdemand-20211218.csv
CAISO-netdemand-20211230.csv
CAISO-netdemand-20201023.csv
CAISO-netdemand-20200329.csv
CAISO-netdemand-20211224.csv
CAISO-netdemand-20211225.csv
CAISO-netdemand-20200328.csv
CAISO-netdemand-20201022.csv
CAISO-netdemand-20211231.csv
CAISO-netdemand-20211219.csv
CAISO-netdemand-20210107.csv
CAISO-netdemand-20200314.csv
CAISO-netdemand-20210113.csv
CAISO-netdemand-20181117.csv
CAISO-netdemand-20181103.csv
CAISO-netdemand-20211027.csv
CAISO-netdemand-20201220.csv
CAISO-netdemand-20200116.csv
CAISO-netdeman

CAISO-netdemand-20181107.csv
CAISO-netdemand-20181113.csv
CAISO-netdemand-20201026.csv
CAISO-netdemand-20211221.csv
CAISO-netdemand-20210117.csv
CAISO-netdemand-20200304.csv
CAISO-netdemand-20211209.csv
CAISO-netdemand-20210103.csv
CAISO-netdemand-20200310.csv
CAISO-netdemand-20200311.csv
CAISO-netdemand-20210102.csv
CAISO-netdemand-20211208.csv
CAISO-netdemand-20200305.csv
CAISO-netdemand-20210116.csv
CAISO-netdemand-20211220.csv
CAISO-netdemand-20201027.csv
CAISO-netdemand-20181112.csv
CAISO-netdemand-20181106.csv
CAISO-netdemand-20200113.csv
CAISO-netdemand-20210314.csv
CAISO-netdemand-20200107.csv
CAISO-netdemand-20201219.csv
CAISO-netdemand-20201231.csv
CAISO-netdemand-20211022.csv
CAISO-netdemand-20210328.csv
CAISO-netdemand-20201225.csv
CAISO-netdemand-20191103.csv
CAISO-netdemand-20190209.csv
CAISO-netdemand-20191117.csv
CAISO-netdemand-20190221.csv
CAISO-netdemand-20210512.csv
CAISO-netdemand-20200701.csv
CAISO-netdemand-20180807.csv
CAISO-netdemand-20180813.csv
CAISO-netdeman

CAISO-netdemand-20180708.csv
CAISO-netdemand-20180720.csv
CAISO-netdemand-20200826.csv
CAISO-netdemand-20190527.csv
CAISO-netdemand-20210214.csv
CAISO-netdemand-20211122.csv
CAISO-netdemand-20210228.csv
CAISO-netdemand-20181210.csv
CAISO-netdemand-20191003.csv
CAISO-netdemand-20181204.csv
CAISO-netdemand-20191017.csv
CAISO-netdemand-20190309.csv
CAISO-netdemand-20190321.csv
CAISO-netdemand-20200211.csv
CAISO-netdemand-20200205.csv
CAISO-netdemand-20201127.csv
CAISO-netdemand-20191201.csv
CAISO-netdemand-20181012.csv
CAISO-netdemand-20220102.csv
CAISO-netdemand-20220116.csv
CAISO-netdemand-20191215.csv
CAISO-netdemand-20181006.csv
CAISO-netdemand-20190123.csv
CAISO-netdemand-20191229.csv
CAISO-netdemand-20190121.csv
CAISO-netdemand-20220128.csv
CAISO-netdemand-20181010.csv
CAISO-netdemand-20191203.csv
CAISO-netdemand-20181004.csv
CAISO-netdemand-20191217.csv
CAISO-netdemand-20190109.csv
CAISO-netdemand-20220114.csv
CAISO-netdemand-20201125.csv
CAISO-netdemand-20200213.csv
CAISO-netdeman

CAISO-netdemand-20190107.csv
CAISO-netdemand-20191219.csv
CAISO-netdemand-20190113.csv
CAISO-netdemand-20220126.csv
CAISO-netdemand-20191225.csv
CAISO-netdemand-20181022.csv
CAISO-netdemand-20191231.csv
CAISO-netdemand-20191209.csv
CAISO-netdemand-20190117.csv
CAISO-netdemand-20190103.csv
CAISO-netdemand-20181026.csv
CAISO-netdemand-20191221.csv
CAISO-netdemand-20220122.csv
CAISO-netdemand-20201107.csv
CAISO-netdemand-20200219.csv
CAISO-netdemand-20201113.csv
CAISO-netdemand-20200225.csv
CAISO-netdemand-20190315.csv
CAISO-netdemand-20181218.csv
CAISO-netdemand-20190301.csv
CAISO-netdemand-20190329.csv
CAISO-netdemand-20181224.csv
CAISO-netdemand-20191023.csv
CAISO-netdemand-20181230.csv
CAISO-netdemand-20210208.csv
CAISO-netdemand-20211116.csv
CAISO-netdemand-20211102.csv
CAISO-netdemand-20210220.csv
CAISO-netdemand-20200812.csv
CAISO-netdemand-20180714.csv
CAISO-netdemand-20190507.csv
CAISO-netdemand-20190513.csv
CAISO-netdemand-20200806.csv
CAISO-netdemand-20180728.csv
CAISO-netdeman

CAISO-netdemand-20200121.csv
CAISO-netdemand-20201217.csv
CAISO-netdemand-20211004.csv
CAISO-netdemand-20200109.csv
CAISO-netdemand-20201203.csv
CAISO-netdemand-20211010.csv
CAISO-netdemand-20181120.csv
CAISO-netdemand-20181108.csv
CAISO-netdemand-20210124.csv
CAISO-netdemand-20201029.csv
CAISO-netdemand-20200323.csv
CAISO-netdemand-20210130.csv
CAISO-netdemand-20210118.csv
CAISO-netdemand-20211206.csv
CAISO-netdemand-20201015.csv
CAISO-netdemand-20211212.csv
CAISO-netdemand-20201001.csv
CAISO-netdemand-20211213.csv
CAISO-netdemand-20201014.csv
CAISO-netdemand-20211207.csv
CAISO-netdemand-20210119.csv
CAISO-netdemand-20210131.csv
CAISO-netdemand-20200322.csv
CAISO-netdemand-20201028.csv
CAISO-netdemand-20210125.csv
CAISO-netdemand-20181109.csv
CAISO-netdemand-20181121.csv
CAISO-netdemand-20211011.csv
CAISO-netdemand-20201202.csv
CAISO-netdemand-20200108.csv
CAISO-netdemand-20211005.csv
CAISO-netdemand-20201216.csv
CAISO-netdemand-20200120.csv
CAISO-netdemand-20210327.csv
CAISO-netdeman